In [1]:
from torchvision.datasets import ImageFolder
from torchvision import transforms
import torch
import numpy as np
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
import pandas as pd
import matplotlib.pyplot as plt
from dataset import tgenerator
from sklearn.dummy import DummyClassifier

seed = 42

np.random.seed(seed)                                                                       
torch.manual_seed(seed) 

img_transform = transforms.Compose([transforms.Resize(255),
                                    transforms.CenterCrop(224),
                                    transforms.ToTensor(),
                                    transforms.Normalize(mean=[0, 0, 0], std=[1, 1, 1])]
                                   )
dataset = ImageFolder(root='../data/train', transform=img_transform)

In [2]:
def getnewmatrix(ds):
    '''
    Takes in all necessary parameters to train the model and returns the model and the loss curves.

    Args:
      num_epochs: Number of epochs to train for 
      dataloader: The training dataloader object that was given in the helper code 
      model: The autoencoder model from the class
      criterion: Loss criterion
      Optimizer: Optimizer to be used 
    Returns:
      model: trained model 
      loss_curve: A list of mean epoch losses over the range of epochs
    '''
    y = []
    imgdat = np.zeros((len(ds), 224 * 224 * 3),dtype=float)
    q = 0

    for i in range(len(ds)):
        y = y + [ds[i][1]]

        im = ds[i][0]
        
        imnew = im.view(-1,im.shape[0] * im.shape[1] * im.shape[2])
        imnew = np.asarray(imnew)
        imgdat[i,] = imnew
        
        if (i % 500 == 0):
            print(i)

    return imgdat, np.asarray(y)

X, y = getnewmatrix(dataset)

0
500
1000
1500
2000
2500
3000
3500
4000
4500


In [3]:
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(X, y)

DummyClassifier(constant=None, random_state=None, strategy='most_frequent')

In [4]:
link = "../data/test"
testset = tgenerator(link)
testset.viewmode()

In [6]:
index = list(range(794))
columns=['file','species']
df_ = pd.DataFrame(index = index, columns=columns)
df_ = df_.fillna(0)

In [7]:
for i in range(794):
    im = testset[i]
    imnew = im.view(-1,im.shape[0] * im.shape[1] * im.shape[2])
    imnew = np.asarray(imnew)
    ytest = dummy_clf.predict(imnew)
    
    if(i % 100 ==0):
        print(i)
    
    df_['file'][i] = testset.list_IDs[i]
    df_['species'][i] = dataset.classes[int(ytest)]
    
df_.to_csv(path_or_buf = "../results/dummypredictions.csv", index = False)

0


/Users/jlakkis/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/Users/jlakkis/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


100
200
300
400
500
600
700
